In [1]:
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import hnswlib
from sklearn.neighbors import NearestNeighbors
from heapq import heappush, heappop

In [2]:

# Path to your GloVe file (update this based on your downloaded version)
glove_path = '/Users/tanishqchaudhari/Desktop/DataScience Proj  datasets/Dataset/glove.6B.100d.txt'

# Load GloVe vectors
word_to_vec = {}
words = []
vectors = []

with open(glove_path, "r", encoding="utf-8") as f:
    for line in f:
        values = line.split()
        word = values[0]  # First token is the word
        vector = np.array(values[1:], dtype=np.float32)  # Rest are vector values
        word_to_vec[word] = vector
        words.append(word)
        vectors.append(vector)

# Convert to numpy array
vectors = np.array(vectors, dtype=np.float32)
print(f"Loaded {len(words)} word vectors of dimension {vectors.shape[1]}")

Loaded 400000 word vectors of dimension 100


In [3]:
from tqdm import tqdm

In [4]:
# Step 1: Generate 10^ random query indices
num_queries = 40000
query_indices = np.random.randint(0, len(words), size=num_queries)

# Step 2: Get the corresponding query vectors
query_vectors = vectors[query_indices]

# Step 3: Use NearestNeighbors to find 100 nearest neighbors
k = 100
nn = NearestNeighbors(n_neighbors=k, algorithm='auto', metric='l2')
nn.fit(vectors)

# Step 4: For each query, get the indices of the 100 nearest neighbors
distances, neighbor_indices = nn.kneighbors(query_vectors)

# Step 5: Store results in a dictionary {query_word: [neighbor_words]}
query_to_neighbors = {}



# Step 5: Store results in a dictionary {query_word: [neighbor_words]}
query_to_neighbors = {}
for i in tqdm(range(len(query_indices)), desc="Finding nearest neighbors"):
    query_idx = query_indices[i]
    query_word = words[query_idx]
    neighbor_words = [words[idx] for idx in neighbor_indices[i]]
    query_to_neighbors[query_word] = neighbor_words

print(f"Stored nearest neighbors for {len(query_to_neighbors)} queries.")


Finding nearest neighbors: 100%|██████████| 40000/40000 [00:00<00:00, 80071.09it/s]

Stored nearest neighbors for 38026 queries.


In [5]:
# import json

# # Load the saved query-to-neighbors mapping
# with open("query_neighbors.json", "r") as f:
#     query_to_neighbors = json.load(f)

# print(f"Loaded {len(query_to_neighbors)} queries from file.")

# # Step 6: Create a dictionary mapping each unique query_word to its vector
# query_word_vectors = {word: word_to_vec[word] for word in query_to_neighbors}

# print(f"Stored vectors for {len(query_word_vectors)} unique query words.")



In [6]:
query_to_neighbors_vectors = {}

for i in tqdm(range(len(query_indices)), desc="Finding nearest neighbors"):
    query_idx = query_indices[i]
    neighbor_indices_list = neighbor_indices[i]
    query_to_neighbors_vectors[query_idx] = neighbor_indices_list  # All indices, not vectors


Finding nearest neighbors: 100%|██████████| 40000/40000 [00:00<00:00, 1600650.29it/s]


In [7]:
from collections import Counter
import pandas as pd

# Flatten all neighbor indices into a single list
all_neighbors = [idx for neighbors in query_to_neighbors_vectors.values() for idx in neighbors]

# Count frequency of each neighbor index
freq_counter = Counter(all_neighbors)

# Create DataFrame and sort
freq_df = pd.DataFrame(freq_counter.items(), columns=["Index", "Frequency"]).sort_values("Frequency", ascending=False)



In [8]:
#printing the average
average = freq_df["Frequency"].mean()
print(f"Average frequency of neighbors: {average}")

Average frequency of neighbors: 11.952186226037322


In [9]:
def compute_hubs(
    neighbor_indices: np.ndarray,
    distances:        np.ndarray,
    freq_df:          pd.DataFrame,
    alpha:            float = None,           # now optional
    method:           str   = "percentile",
    threshold:        float = 95.0,
    top_k:            int   = None
) -> (pd.Series, set):
    """Compute combined hub‐scores f_i and select hub items H."""

    # 1) Normalize frequency
    freq_series = freq_df.set_index("Index")["Frequency"].astype(float)
    c_min, c_max = freq_series.min(), freq_series.max()
    if c_max > c_min:
        f_freq = (freq_series - c_min) / (c_max - c_min)
    else:
        f_freq = pd.Series(0.0, index=freq_series.index)

    # 2) Compute and normalize average distance (then flip)
    df = pd.DataFrame({
        "Index":    neighbor_indices.ravel(),
        "Distance": distances.ravel()
    })
    d_bar = df.groupby("Index")["Distance"].mean()
    d_min, d_max = d_bar.min(), d_bar.max()
    if d_max > d_min:
        d_tilde = (d_bar - d_min) / (d_max - d_min)
    else:
        d_tilde = pd.Series(0.0, index=d_bar.index)
    f_dist = 1.0 - d_tilde

    # 3) Align indices
    common = f_freq.index.intersection(f_dist.index)
    f_f = f_freq.loc[common]
    f_d = f_dist.loc[common]

    # 4) Auto‐compute alpha if not provided
    if alpha is None:
        var_f = f_f.var()
        var_d = f_d.var()
        total = var_f + var_d
        alpha = var_f / total if total > 0 else 0.5

    # 5) Combine
    combined = alpha * f_f + (1 - alpha) * f_d
    combined.name = "f_i"

    # 6) Select hubs
    if method == "percentile":
        cutoff = np.percentile(combined.values, threshold)
        hub_set = set(combined[combined >= cutoff].index)

    elif method == "stddev":
        mu, sigma = combined.mean(), combined.std()
        cutoff = mu + threshold * sigma
        hub_set = set(combined[combined >= cutoff].index)

    elif method == "top_k":
        if top_k is None:
            raise ValueError("top_k must be provided when method='top_k'")
        hub_set = set(combined.nlargest(top_k).index)

    else:
        raise ValueError(f"unknown method {method!r}")

    return combined, hub_set


In [10]:
combined_scores, hubs = compute_hubs(
    neighbor_indices,
    distances,
    freq_df,
    alpha=None,
    method="percentile",
    threshold=90.0
)

In [11]:
import numpy as np
from heapq import heappush, heappop
import random
import math

# HNSW CLASS

# Obj Of HNSW CLASS

In [12]:
def _distance(self, i, j):
    return np.linalg.norm(self.vectors[i] - self.vectors[j])


In [13]:
import numpy as np
from heapq import heappush, heappop
import random
import math

class HNSW:
    """
    Standard HNSW implementation based on the provided code [1],
    with hub-related modifications removed.
    """
    def __init__(self, dim, max_elements, M=16, ef_construction=200):
        """
        Initializes the HNSW index.

        Args:
            dim (int): Dimensionality of the vectors.
            max_elements (int): Estimated maximum number of elements (informational).
            M (int): Maximum number of connections per node per layer.
            ef_construction (int): Size of the dynamic candidate list during construction.
        """
        self.dim = dim
        self.max_elements = max_elements # Informational
        self.M = M
        self.ef_construction = ef_construction

        # Store graph layers: layers[l] is a dict {node_idx: [neighbor_indices]}
        self.layers = []
        # Store the actual vectors, index corresponds to node_idx
        self.vectors = []
        # Track the entry point (index of the node in the highest layer)
        self.entry_point = None
        # Precompute layer multiplier
        self.ml = 1 / math.log(self.M) if self.M > 1 else 1


    def _get_layer(self):
        """
        Determine the layer for a new node based on exponential decay
        using the precomputed multiplier (mL).

        Returns:
            int: The selected layer index (>= 0).
        """
        # Calculate layer using the standard HNSW formula
        layer = max(0, int(-math.log(random.random()) * self.ml))
        return layer

    # --- _distance method remains unchanged ---
    def _distance(self, idx1, idx2):
        """Calculate Euclidean distance between two vectors by index."""
        if not (0 <= idx1 < len(self.vectors) and 0 <= idx2 < len(self.vectors)):
            # Handle cases where one or both indices are out of bounds
            # This might happen during pruning if a node was considered but
            # doesn't actually exist in the current vector list state.
            return float('inf')
        try:
            return np.linalg.norm(self.vectors[idx1] - self.vectors[idx2])
        except IndexError:
             # Should ideally not happen if bounds check passes, but as extra safety
             return float('inf')


    # --- _search_layer method remains unchanged ---
    # (Includes robustness checks from original code [1])
    def _search_layer(self, query_vec, layer_idx, ep_idx, ef):
        """Search within a specific layer starting from an entry point."""
        # Basic layer validity check
        if layer_idx < 0 or layer_idx >= len(self.layers):
            return []
        layer_graph = self.layers[layer_idx]
        if not layer_graph: # Layer exists but is empty
            return []

        # Robustness: Check if entry point is valid for this layer and vector list
        if ep_idx not in layer_graph or ep_idx >= len(self.vectors):
            # If invalid, try to find a random valid node in the layer as a fallback EP
            try:
                valid_indices = [idx for idx in layer_graph if idx < len(self.vectors)]
                if not valid_indices:
                    return [] # No valid nodes in this layer
                ep_idx = random.choice(valid_indices) # Choose a random valid node
            except IndexError: # Should not happen if valid_indices check passed
                return []

        visited = set()
        candidates = [] # Min-heap: (distance, node_idx)
        results = []    # Max-heap: (-distance, node_idx)

        # Initialize search with the entry point
        try:
             # Ensure vector exists before calculating distance
             if ep_idx >= len(self.vectors):
                 # This case should be caught by the initial check, but for safety:
                 return []
             initial_dist = np.linalg.norm(query_vec - self.vectors[ep_idx])
             heappush(candidates, (initial_dist, ep_idx))
             heappush(results, (-initial_dist, ep_idx))
             visited.add(ep_idx)
        except IndexError:
             # Fallback if vector access fails unexpectedly
             # print(f"Warning: IndexError accessing vector for entry point {ep_idx}...") # Optional warning
             return []

        # Greedy search loop
        while candidates:
            try:
                dist_candidate, current_idx = heappop(candidates)
            except IndexError: # Heap is empty
                break

            # Get distance of the farthest node found so far
            farthest_dist_in_results = -results[0][0] if results else float('inf')

            # Early termination condition
            if dist_candidate > farthest_dist_in_results and len(results) >= ef:
                 break # All remaining candidates are farther than the worst result

            # Check if current node is still valid (might be needed with concurrent modifications, less so here)
            # Also check if it exists in the current layer graph
            if current_idx not in layer_graph or current_idx >= len(self.vectors):
                continue # Skip if node became invalid or is not in this layer

            # Explore neighbors
            for neighbor_idx in layer_graph.get(current_idx, []):
                 # Check if neighbor is valid and not visited
                 if neighbor_idx not in visited and neighbor_idx < len(self.vectors):
                    visited.add(neighbor_idx)
                    try:
                        dist_neighbor = np.linalg.norm(query_vec - self.vectors[neighbor_idx])
                    except IndexError:
                        continue # Skip if neighbor vector access fails

                    # Get updated farthest distance
                    farthest_dist_in_results = -results[0][0] if results else float('inf')

                    # If neighbor is closer than farthest result or results list is not full
                    if dist_neighbor < farthest_dist_in_results or len(results) < ef:
                        # Add to results (maintaining max-heap property)
                        heappush(results, (-dist_neighbor, neighbor_idx))
                        # If results exceed ef, remove the farthest
                        if len(results) > ef:
                            heappop(results)
                        # Add neighbor to candidates for further exploration
                        heappush(candidates, (dist_neighbor, neighbor_idx))

        # Convert max-heap results to sorted list (distance, idx)
        final_results = sorted([(-d, idx) for d, idx in results])
        return final_results[:ef] # Return top ef results


    def insert(self, vector):
        """Insert a vector into the HNSW index."""
        new_idx = len(self.vectors) # Index for the new vector
        self.vectors.append(np.array(vector)) # Store the vector

        # Determine the layers the new node will belong to
        node_max_layer = self._get_layer() # Use standard layer selection

        current_ep_idx = self.entry_point

        # Ensure enough layers exist in the graph structure
        while len(self.layers) <= node_max_layer:
            self.layers.append(dict()) # Add new empty layer dictionaries

        # --- Rest of insert method unchanged from original code [1] ---
        # (Includes robustness checks)
        current_insert_vec = self.vectors[new_idx] # Use the newly added vector

        # Phase 1: Find entry points in upper layers (down to node_max_layer + 1)
        for l in range(len(self.layers) - 1, node_max_layer, -1):
             if current_ep_idx is None: break # Cannot proceed without an entry point
             if not self.layers[l]: continue # Skip empty layers

             # Ensure entry point validity before search in this layer
             if current_ep_idx not in self.layers[l] or current_ep_idx >= len(self.vectors):
                 try:
                    # Fallback: Find a random valid node in the layer
                    valid_indices = [idx for idx in self.layers[l] if idx < len(self.vectors)]
                    if not valid_indices: continue # Skip layer if no valid nodes
                    current_ep_idx = random.choice(valid_indices)
                 except IndexError: continue # Should not happen

             # Search for the closest node in layer l to the new vector (ef=1)
             search_results = self._search_layer(current_insert_vec, l, current_ep_idx, ef=1)
             if search_results:
                 # Update the entry point for the next lower layer
                 found_ep_idx = search_results[0][1]
                 # Check validity before assignment
                 if found_ep_idx < len(self.vectors):
                      current_ep_idx = found_ep_idx

        # Phase 2: Insert node in layers node_max_layer down to 0
        for l in range(min(node_max_layer, len(self.layers) - 1), -1, -1):
            layer_graph = self.layers[l]

            # Ensure valid entry point for the search in this layer
            if current_ep_idx is None or current_ep_idx not in layer_graph or current_ep_idx >= len(self.vectors):
                 if layer_graph: # If the layer is not empty
                     try:
                         # Fallback: Find a random valid node
                         valid_indices = [idx for idx in layer_graph if idx < len(self.vectors)]
                         if not valid_indices: current_ep_idx = None # No valid EPs possible
                         else: current_ep_idx = random.choice(valid_indices)
                     except IndexError: current_ep_idx = None
                 else: current_ep_idx = None # Layer is empty, no EP

            # Find neighbors using search_layer with ef_construction
            if current_ep_idx is None:
                 neighbors = [] # No entry point, cannot search
            else:
                 # Search for ef_construction nearest neighbors
                 neighbors = self._search_layer(current_insert_vec, l, current_ep_idx, self.ef_construction)
                 # Update entry point for next layer (closest neighbor found)
                 if neighbors:
                      found_ep_idx = neighbors[0][1]
                      # Check validity before assignment
                      if found_ep_idx < len(self.vectors):
                           current_ep_idx = found_ep_idx

            # Select M best neighbors based on distance
            connections = [idx for dist, idx in neighbors[:self.M]]
            # Add connections for the new node in this layer
            layer_graph[new_idx] = connections

            # Add backlinks from neighbors to the new node, maintaining M limit
            for neighbor_idx in connections:
                 # Ensure neighbor exists and is valid before modifying its connections
                 if neighbor_idx in layer_graph and neighbor_idx < len(self.vectors):
                     neighbor_connections = layer_graph[neighbor_idx]
                     if new_idx not in neighbor_connections: # Avoid duplicate connections
                          neighbor_connections.append(new_idx)
                          # Prune connections if exceeding M
                          if len(neighbor_connections) > self.M:
                               # Ensure all connections are valid before distance calculation for pruning
                               valid_conns = [cidx for cidx in neighbor_connections if cidx < len(self.vectors)]
                               if len(valid_conns) < len(neighbor_connections):
                                   # If some connections became invalid, just update the list
                                   layer_graph[neighbor_idx] = valid_conns
                                   # Re-check length; might not need pruning anymore
                                   if len(valid_conns) <= self.M:
                                        continue # Skip pruning if count is now okay

                               # Calculate distances only for valid connections
                               distances = []
                               for conn_idx in valid_conns:
                                   dist = self._distance(neighbor_idx, conn_idx)
                                   if dist != float('inf'): # Only consider valid distances
                                       distances.append((dist, conn_idx))

                               # Sort by distance and keep the M closest
                               distances.sort()
                               layer_graph[neighbor_idx] = [idx for dist, idx in distances[:self.M]]

        # Update the global entry point if the new node is in a higher layer
        current_ep_layer = self._get_node_layer(self.entry_point) # Find current EP's highest layer
        if self.entry_point is None or node_max_layer > current_ep_layer:
            self.entry_point = new_idx


    # --- Helper method to find the highest layer a node exists in ---
    def _get_node_layer(self, node_idx):
        """Helper to find the highest layer index a node exists in."""
        if node_idx is None or node_idx < 0: return -1
        for l in range(len(self.layers) - 1, -1, -1):
             # Check if layer exists and node is in the layer's keys
             if l < len(self.layers) and node_idx in self.layers[l]:
                 return l
        return -1 # Node not found in any layer


    # --- search method remains unchanged ---
    # (Includes robustness checks from original code [1])
    def search(self, query_vec, k=10):
        """Search for the k nearest neighbors of query_vec."""
        # Determine ef for search (at least k)
        ef_search = max(self.ef_construction, k)
        current_ep_idx = self.entry_point

        # Check if entry point is valid at the start
        if current_ep_idx is None or current_ep_idx >= len(self.vectors):
            #print("Warning: Entry point is None or invalid at search start.") # Optional warning
            # Potentially try to find *any* valid node if layers exist?
            # For now, return empty if no valid starting point.
            return []

        query_vec = np.array(query_vec) # Ensure query is a numpy array

        # Phase 1: Navigate upper layers (down to layer 1) to find good entry point for layer 0
        for l in range(len(self.layers) - 1, 0, -1): # Stop at layer 1
            if not self.layers[l]: continue # Skip empty layers

            # Ensure current entry point is valid for this layer
            if current_ep_idx not in self.layers[l] or current_ep_idx >= len(self.vectors):
                 try:
                      # Fallback: Find a random valid node in the layer
                      valid_indices = [idx for idx in self.layers[l] if idx < len(self.vectors)]
                      if not valid_indices: continue # Skip layer if no valid nodes
                      current_ep_idx = random.choice(valid_indices)
                 except IndexError: continue # Should not happen

            # Search layer l for the closest node (ef=1)
            search_results = self._search_layer(query_vec, l, current_ep_idx, ef=1)
            if search_results:
                 # Update entry point for the next lower layer
                 found_ep_idx = search_results[0][1]
                 # Check validity before assignment
                 if found_ep_idx < len(self.vectors):
                      current_ep_idx = found_ep_idx

        # Phase 2: Search layer 0 using the refined entry point
        # Ensure the final entry point for layer 0 is valid before the final search
        if current_ep_idx >= len(self.vectors) or \
           (0 < len(self.layers) and current_ep_idx not in self.layers[0]):
             # If layer 0 exists and has nodes, try to find a fallback EP within it
             if 0 < len(self.layers) and self.layers[0]:
                  try:
                      valid_indices_l0 = [idx for idx in self.layers[0] if idx < len(self.vectors)]
                      if not valid_indices_l0: return [] # No valid nodes in layer 0
                      current_ep_idx = random.choice(valid_indices_l0)
                  except IndexError: return [] # Should not happen
             else: return [] # Layer 0 doesn't exist or is empty

        # Perform final search in layer 0 with ef_search
        neighbors = self._search_layer(query_vec, 0, current_ep_idx, ef_search)

        # Return the top k results by index
        return [idx for dist, idx in neighbors[:k]]



In [14]:
import numpy as np
from heapq import heappush, heappop
import random
import math

class HNSWfreqdistance:
    """
    HNSW implementation based on paste.txt [1], modified to boost layer
    probability for specified 'hub' nodes during insertion.
    Search performance remains identical to the base HNSW implementation.
    """
    def __init__(self, dim, max_elements, hubs, boost_const, M=16, ef_construction=200):
        """
        Initializes the HNSWfreqdistance index.

        Args:
            dim (int): Dimensionality of the vectors.
            max_elements (int): Estimated maximum number of elements (informational).
            hubs (set): A set of integer indices representing 'hub' nodes.
                        Nodes whose *future* index is in this set will have
                        their layer assignment probability boosted.
            boost_const (float): A small constant added to the layer calculation
                                 multiplier (mL) for hub nodes.
            M (int): Maximum number of connections per node per layer.
            ef_construction (int): Size of the dynamic candidate list during construction.
        """
        self.dim = dim
        self.max_elements = max_elements # Informational
        self.M = M
        self.ef_construction = ef_construction
        self.hubs = hubs if hubs is not None else set() # Ensure hubs is a set
        self.boost_const = boost_const # Constant for boosting hub layer probability

        # Store graph layers: layers[l] is a dict {node_idx: [neighbor_indices]}
        self.layers = []
        # Store the actual vectors, index corresponds to node_idx
        self.vectors = []
        # Track the entry point (index of the node in the highest layer)
        self.entry_point = None
        # Precompute layer multiplier
        self.ml = 1 / math.log(self.M) if self.M > 1 else 1


    # *** MODIFIED to accept node_idx and apply boost ***
    def _get_layer(self, node_idx):
        """
        Determine the layer for a new node based on exponential decay.
        If node_idx is in self.hubs, boost the probability of higher layers.

        Args:
            node_idx (int): The index the new node *will* have upon insertion.

        Returns:
            int: The selected layer index (>= 0).
        """
        # Use precomputed base multiplier
        current_ml = self.ml

        # Apply boost if the node index is designated as a hub
        if node_idx in self.hubs:
            current_ml += self.boost_const

        # Calculate layer using the (potentially boosted) multiplier
        layer = max(0, int(-math.log(random.random()) * current_ml))
        return layer

    # --- _distance method remains unchanged from paste.txt [1] ---
    def _distance(self, idx1, idx2):
        if not (0 <= idx1 < len(self.vectors) and 0 <= idx2 < len(self.vectors)):
            return float('inf')
        return np.linalg.norm(self.vectors[idx1] - self.vectors[idx2])

    # --- _search_layer method remains unchanged from paste.txt [1] ---
    # (Includes robustness checks from paste.txt [1])
    def _search_layer(self, query_vec, layer_idx, ep_idx, ef):
        if layer_idx < 0 or layer_idx >= len(self.layers): return []
        layer_graph = self.layers[layer_idx]
        if not layer_graph: return []

        if ep_idx not in layer_graph or ep_idx >= len(self.vectors):
            try:
                valid_indices = [idx for idx in layer_graph if idx < len(self.vectors)]
                if not valid_indices: return []
                ep_idx = random.choice(valid_indices)
            except IndexError: return []

        visited = set()
        candidates = []
        results = []

        try:
            if ep_idx >= len(self.vectors): return []
            initial_dist = np.linalg.norm(query_vec - self.vectors[ep_idx])
            heappush(candidates, (initial_dist, ep_idx))
            heappush(results, (-initial_dist, ep_idx))
            visited.add(ep_idx)
        except IndexError:
             # print(f"Warning: IndexError accessing vector for entry point {ep_idx}...") # Optional warning
             return []

        while candidates:
            try:
                dist_candidate, current_idx = heappop(candidates)
            except IndexError: break

            farthest_dist_in_results = -results[0][0] if results else float('inf')
            if dist_candidate > farthest_dist_in_results and len(results) >= ef:
                 break

            if current_idx not in layer_graph or current_idx >= len(self.vectors):
                continue

            for neighbor_idx in layer_graph.get(current_idx, []):
                if neighbor_idx not in visited and neighbor_idx < len(self.vectors):
                    visited.add(neighbor_idx)
                    dist_neighbor = np.linalg.norm(query_vec - self.vectors[neighbor_idx])
                    farthest_dist_in_results = -results[0][0] if results else float('inf')

                    if dist_neighbor < farthest_dist_in_results or len(results) < ef:
                        heappush(results, (-dist_neighbor, neighbor_idx))
                        if len(results) > ef: heappop(results)
                        heappush(candidates, (dist_neighbor, neighbor_idx))

        final_results = sorted([(-d, idx) for d, idx in results])
        return final_results[:ef]

    # *** MODIFIED to pass new_idx to _get_layer ***
    def insert(self, vector):
        """Insert a vector into the HNSW index."""
        new_idx = len(self.vectors) # Determine index before layer calculation
        self.vectors.append(np.array(vector))

        # *** MODIFIED CALL ***: Pass the node's future index to _get_layer
        node_max_layer = self._get_layer(new_idx)

        current_ep_idx = self.entry_point

        while len(self.layers) <= node_max_layer:
            self.layers.append(dict())

        # --- Rest of insert method unchanged from paste.txt [1] ---
        # (Includes robustness checks)
        current_insert_vec = self.vectors[new_idx] # Use stored vector for search

        for l in range(len(self.layers) - 1, node_max_layer, -1):
             if current_ep_idx is None: break
             if not self.layers[l]: continue

             # Ensure entry point validity before search
             if current_ep_idx not in self.layers[l] or current_ep_idx >= len(self.vectors):
                 try:
                    valid_indices = [idx for idx in self.layers[l] if idx < len(self.vectors)]
                    if not valid_indices: continue
                    current_ep_idx = random.choice(valid_indices)
                 except IndexError: continue

             search_results = self._search_layer(current_insert_vec, l, current_ep_idx, ef=1)
             if search_results:
                 found_ep_idx = search_results[0][1]
                 if found_ep_idx < len(self.vectors): # Check validity before assignment
                      current_ep_idx = found_ep_idx

        for l in range(min(node_max_layer, len(self.layers) - 1), -1, -1):
            layer_graph = self.layers[l]

            # Ensure valid entry point for search
            if current_ep_idx is None or current_ep_idx not in layer_graph or current_ep_idx >= len(self.vectors):
                 if layer_graph:
                     try:
                         valid_indices = [idx for idx in layer_graph if idx < len(self.vectors)]
                         if not valid_indices: current_ep_idx = None
                         else: current_ep_idx = random.choice(valid_indices)
                     except IndexError: current_ep_idx = None
                 else: current_ep_idx = None

            if current_ep_idx is None:
                 neighbors = []
            else:
                 neighbors = self._search_layer(current_insert_vec, l, current_ep_idx, self.ef_construction)
                 if neighbors:
                      found_ep_idx = neighbors[0][1]
                      if found_ep_idx < len(self.vectors): # Check validity
                           current_ep_idx = found_ep_idx

            connections = [idx for dist, idx in neighbors[:self.M]]
            layer_graph[new_idx] = connections

            for neighbor_idx in connections:
                 # Check validity before adding backlink
                 if neighbor_idx in layer_graph and neighbor_idx < len(self.vectors):
                     neighbor_connections = layer_graph[neighbor_idx]
                     if new_idx not in neighbor_connections:
                          neighbor_connections.append(new_idx)
                          if len(neighbor_connections) > self.M:
                               # Check connection validity before distance calc for pruning
                               valid_conns = [cidx for cidx in neighbor_connections if cidx < len(self.vectors)]
                               if len(valid_conns) < len(neighbor_connections):
                                   layer_graph[neighbor_idx] = valid_conns
                                   continue

                               distances = [(self._distance(neighbor_idx, conn_idx), conn_idx) for conn_idx in valid_conns]
                               distances.sort()
                               layer_graph[neighbor_idx] = [idx for dist, idx in distances[:self.M]]

        # Update global entry point logic (unchanged from paste.txt [1])
        current_ep_layer = self._get_node_layer(self.entry_point) # Use helper
        if self.entry_point is None or node_max_layer > current_ep_layer:
            self.entry_point = new_idx


    # --- Helper method from paste-2.txt [2] added for clarity ---
    def _get_node_layer(self, node_idx):
        # Helper to find highest layer a node exists in
        if node_idx is None or node_idx < 0: return -1
        for l in range(len(self.layers) - 1, -1, -1):
             if node_idx in self.layers[l]:
                 return l
        return -1 # Node not found

    # --- search method remains unchanged from paste.txt [1] ---
    # (Includes robustness checks)
    def search(self, query_vec, k=10):
        ef_search = max(self.ef_construction, k)
        current_ep_idx = self.entry_point

        if current_ep_idx is None or current_ep_idx >= len(self.vectors):
            #print("Warning: Entry point is None or invalid...") # Optional
            return []

        query_vec = np.array(query_vec)

        for l in range(len(self.layers) - 1, 0, -1): # Down to layer 1
            if not self.layers[l]: continue

            # Ensure entry point validity
            if current_ep_idx not in self.layers[l] or current_ep_idx >= len(self.vectors):
                 try:
                     valid_indices = [idx for idx in self.layers[l] if idx < len(self.vectors)]
                     if not valid_indices: continue
                     current_ep_idx = random.choice(valid_indices)
                 except IndexError: continue

            search_results = self._search_layer(query_vec, l, current_ep_idx, ef=1)
            if search_results:
                 found_ep_idx = search_results[0][1]
                 if found_ep_idx < len(self.vectors): # Check validity
                      current_ep_idx = found_ep_idx

        # Ensure final entry point for layer 0 is valid
        if current_ep_idx >= len(self.vectors) or (0 < len(self.layers) and current_ep_idx not in self.layers[0]):
             if 0 < len(self.layers) and self.layers[0]:
                  try:
                      valid_indices_l0 = [idx for idx in self.layers[0] if idx < len(self.vectors)]
                      if not valid_indices_l0: return []
                      current_ep_idx = random.choice(valid_indices_l0)
                  except IndexError: return []
             else: return []

        neighbors = self._search_layer(query_vec, 0, current_ep_idx, ef_search)
        return [idx for dist, idx in neighbors[:k]]



In [15]:
import time

In [24]:
# --- User parameters ---
K = 100                   # Number of neighbors for recall
NUM_QUERIES = 40000       # Number of queries to use


DIMENSION = vectors.shape[1]
NUM_ELEMENTS = vectors.shape[0]

# --- Select query vectors ---
idxs = np.random.choice(NUM_ELEMENTS, size=min(40000, NUM_ELEMENTS), replace=False)
query_vectors = vectors[idxs]
num_actual_queries = len(query_vectors)

# --- Helper functions ---
def calculate_recall(ground_truth, found, k):
    total_found = 0
    num_q = len(ground_truth)
    expected = num_q * k

    for i in range(num_q):
        gt_set = set(ground_truth[i][:k])
        res = found[i]
        found_set = set(res[:k]) if res is not None else set()
        total_found += len(gt_set & found_set)

    return total_found / expected if expected > 0 else 1.0

# --- Ground Truth Calculation (Corrected) ---
import time # Ensure time is imported if not already globally
from sklearn.neighbors import NearestNeighbors # Ensure imported
import numpy as np # Ensure imported

def compute_ground_truth(data, queries, k):
    """
    Computes ground truth returning both distances and indices.
    """
    print(f"Computing ground truth with brute-force for {len(queries)} queries and k={k}...")
    t0 = time.perf_counter()

    # Ensure k is valid for the number of data points
    # We need at least k+1 points to find k neighbors (excluding the point itself if it's in data)
    # However, kneighbors handles finding fewer if necessary. Let's check k against shape[0].
    if k >= data.shape[0]:
        print(f"Warning: k={k} is >= number of data points ({data.shape[0]}). Reducing k to {data.shape[0] - 1}.")
        k = max(1, data.shape[0] - 1) # Find at least 1 neighbor if possible

    if k <= 0:
       print("Error: Cannot compute neighbors with k <= 0.")
       return np.array([]), np.array([]) # Return empty arrays

    nn = NearestNeighbors(n_neighbors=k, algorithm='brute', metric='euclidean')
    nn.fit(data)

    # Capture BOTH distances and indices from kneighbors
    gt_distances, gt_indices = nn.kneighbors(queries, n_neighbors=k)

    t1 = time.perf_counter()
    print(f"Done computing ground truth in {t1 - t0:.2f}s")

    # Return BOTH distances and indices
    return gt_distances, gt_indices

# --- Evaluation for base HNSW ---
def evaluate_hnsw_base(all_vecs, queries, gt, k, M=16, ef_const=200):
    print("\nEvaluating HNSW (base)")
    index = HNSW(dim=all_vecs.shape[1], max_elements=all_vecs.shape[0], M=M, ef_construction=ef_const)

    # Indexing with progress bar
    t0 = time.perf_counter()
    for v in tqdm(all_vecs, total=all_vecs.shape[0], desc="HNSW (base) indexing"):
        index.insert(v)
    t1 = time.perf_counter()

    # Querying with progress bar
    results = []
    t_q0 = time.perf_counter()
    for q in tqdm(queries, total=len(queries), desc="HNSW (base) querying"):
        results.append(index.search(q, k=k))
    t_q1 = time.perf_counter()

    recall = calculate_recall(gt, results, k)
    total_q = t_q1 - t_q0
    avg_q = total_q / len(queries)

    print(f"Index time: {t1 - t0:.2f}s")
    print(f"Total query time: {total_q:.2f}s, Avg/query: {avg_q*1000:.2f}ms")
    print(f"Recall@{k}: {recall:.4f}")

    return (t1 - t0), total_q, avg_q, recall



In [21]:
# Add this new helper function (e.g., after calculate_recall)

def calculate_recall_dist(gt_distances, found_results, k):
    """
    Calculates recall based on distance.

    For each query, determines the distance to the k-th ground truth neighbor (radius).
    Counts how many of the k found neighbors fall within this radius.
    Averages this count over all queries and divides by k.

    Args:
        gt_distances (np.ndarray): Array of shape (num_queries, k) containing
                                   distances to ground truth neighbors.
        found_results (list): List of lists. Each inner list contains
                              tuples (distance, index) for found neighbors.
        k (int): Number of neighbors considered.

    Returns:
        float: The distance-based recall score.
    """
    total_within_radius = 0
    num_queries = len(gt_distances)

    if num_queries == 0 or k == 0:
        return 1.0 # Or 0.0, depending on definition for empty cases

    for i in range(num_queries):
        if i >= len(found_results) or not found_results[i]:
            continue # Skip if no results found for this query

        # Ground truth distance for the k-th neighbor (defines the radius)
        # Ensure k-1 is a valid index
        if k > gt_distances.shape[1]:
             print(f"Warning: k={k} exceeds ground truth neighbors found ({gt_distances.shape[1]})")
             continue # Or handle differently
        radius = gt_distances[i, k-1]

        # Distances of the *found* neighbors (up to k)
        # found_results[i] is like [(dist1, idx1), (dist2, idx2), ...]
        found_dists = [dist for dist, idx in found_results[i][:k]]

        # Count how many found neighbors are within the radius
        count_within = sum(1 for dist in found_dists if dist <= radius)
        total_within_radius += count_within

    # Average recall: (total found within radius) / (total possible = num_queries * k)
    expected = num_queries * k
    return total_within_radius / expected if expected > 0 else 1.0

In [28]:
# --- Evaluation for base HNSW (Modified to work with index-only search results) ---
def evaluate_hnsw_base(all_vecs, queries, gt_distances, gt_indices, k, M=16, ef_const=200):
    print("\nEvaluating HNSW (base)")
    # Assume index object is already created and populated elsewhere if we skip indexing
    # For this specific request, we NEED to run indexing if the index isn't already built and available globally
    # If index IS already built, we'd load/reuse it. Assuming here we run the whole process.
    index = HNSW(dim=all_vecs.shape[1], max_elements=all_vecs.shape[0], M=M, ef_construction=ef_const)

    t0 = time.perf_counter()
    # Check if index is already populated (e.g., if run previously)
    # This is tricky without proper state management; let's assume re-indexing for now
    # If you have a pre-built index object, you'd skip this loop.
    # For this example, we still show the indexing time calculation.
    if not index.vectors: # Simple check if index appears empty
         print("Index seems empty, running indexing...")
         for v in tqdm(all_vecs, total=all_vecs.shape[0], desc="HNSW (base) indexing"):
             index.insert(v)
    else:
        print("Skipping indexing, assuming index is pre-built.") # Placeholder message
    t1 = time.perf_counter()
    index_time = t1 - t0 # Record indexing time even if skipped conceptually

    # Querying with progress bar - Gets lists of indices
    results_indices = [] # Store results from index.search (list of lists of indices)
    t_q0 = time.perf_counter()
    for q in tqdm(queries, total=len(queries), desc="HNSW (base) querying"):
        results_indices.append(index.search(q, k=k)) # Returns list[int]
    t_q1 = time.perf_counter()
    query_time = t_q1 - t_q0

    # --- Post-processing for recall calculations ---

    # 1. Standard Recall: Uses the indices directly
    # results_indices is already [[idx1, idx2,...], [idx3, idx4,...], ...]
    recall_std = calculate_recall(gt_indices, results_indices, k)

    # 2. Distance Recall: Manually compute distances for found indices
    results_with_distances = [] # Will store [[(d1,i1), (d2,i2),..], ...]
    for i, q_vec in enumerate(queries):
        found_idxs = results_indices[i] # Get the indices found for this query
        query_results_dist = []
        for idx in found_idxs:
            if 0 <= idx < len(all_vecs): # Check validity
                # Calculate distance between query and the vector at found index
                dist = np.linalg.norm(q_vec - all_vecs[idx])
                query_results_dist.append((dist, idx))
            else:
                 # Handle case where index might be invalid (shouldn't happen often)
                 query_results_dist.append((float('inf'), idx))
        # Sort by distance just in case search didn't guarantee order (though it should)
        query_results_dist.sort(key=lambda x: x[0])
        results_with_distances.append(query_results_dist)

    recall_dist = calculate_recall_dist(gt_distances, results_with_distances, k)

    avg_q = query_time / len(queries) if len(queries) > 0 else 0

    print(f"Index time: {index_time:.2f}s") # Show calculated/placeholder index time
    print(f"Total query time: {query_time:.2f}s, Avg/query: {avg_q*1000:.2f}ms")
    print(f"Recall@{k} (std): {recall_std:.4f}")
    print(f"Recall@{k} (dist): {recall_dist:.4f}")

    return index_time, query_time, avg_q, recall_std, recall_dist


# --- Evaluation for HNSWfreqdistance (Modified to work with index-only search results) ---
def evaluate_hnsw_freq(all_vecs, queries, gt_distances, gt_indices, k, hubs, boost_scaling_factor, M=16, ef_const=200):
    print("\nEvaluating HNSWfreqdistance")

    # Dynamic boost calculation (remains the same)
    if all_vecs.shape[0] > 0 and hubs is not None:
        hub_ratio = len(hubs) / all_vecs.shape[0]
        boost_const = boost_scaling_factor * hub_ratio
        print(f"  Hub ratio: {hub_ratio:.4f}, Calculated boost_const: {boost_const:.4f}")
    else:
        boost_const = 0
        print("  Hub ratio: N/A or 0, boost_const set to 0")

    # Again, assuming index needs to be built or reused correctly.
    index = HNSWfreqdistance(dim=all_vecs.shape[1], max_elements=all_vecs.shape[0],
                              hubs=hubs, boost_const=boost_const,
                              M=M, ef_construction=ef_const)

    t0 = time.perf_counter()
    # Similar check/assumption about indexing as in evaluate_hnsw_base
    if not index.vectors:
         print("Index seems empty, running indexing...")
         for v in tqdm(all_vecs, total=all_vecs.shape[0], desc="HNSWfreqdistance indexing"):
             index.insert(v)
    else:
         print("Skipping indexing, assuming index is pre-built.")
    t1 = time.perf_counter()
    index_time = t1 - t0

    # Querying - Gets lists of indices
    results_indices = []
    t_q0 = time.perf_counter()
    for q in tqdm(queries, total=len(queries), desc="HNSWfreqdistance querying"):
        results_indices.append(index.search(q, k=k)) # Returns list[int]
    t_q1 = time.perf_counter()
    query_time = t_q1 - t_q0

    # --- Post-processing for recall calculations ---

    # 1. Standard Recall
    recall_std = calculate_recall(gt_indices, results_indices, k)

    # 2. Distance Recall (Manual distance calculation)
    results_with_distances = []
    for i, q_vec in enumerate(queries):
        found_idxs = results_indices[i]
        query_results_dist = []
        for idx in found_idxs:
             if 0 <= idx < len(all_vecs):
                dist = np.linalg.norm(q_vec - all_vecs[idx])
                query_results_dist.append((dist, idx))
             else:
                query_results_dist.append((float('inf'), idx))
        query_results_dist.sort(key=lambda x: x[0])
        results_with_distances.append(query_results_dist)

    recall_dist = calculate_recall_dist(gt_distances, results_with_distances, k)

    avg_q = query_time / len(queries) if len(queries) > 0 else 0

    print(f"Index time: {index_time:.2f}s")
    print(f"Total query time: {query_time:.2f}s, Avg/query: {avg_q*1000:.2f}ms")
    print(f"Recall@{k} (std): {recall_std:.4f}")
    print(f"Recall@{k} (dist): {recall_dist:.4f}")

    return index_time, query_time, avg_q, recall_std, recall_dist

# HNSW with HUB BOOSTING

In [29]:
# In the cell with "Main Comparison" (Cell 19 originally)
# --- Main Comparison ---
if __name__ == '__main__':
    # K and NUM_QUERIES defined earlier
    K = 100
    NUM_QUERIES = 40000 # Reduced from 40000 for potentially faster testing, adjust as needed

    # Select query vectors (make sure this uses the NUM_QUERIES variable)
    # Ensure we don't request more queries than available vectors
    actual_num_queries = min(NUM_QUERIES, vectors.shape[0])
    if actual_num_queries < NUM_QUERIES:
        print(f"Warning: Requested {NUM_QUERIES} queries, but only {vectors.shape[0]} vectors available. Using {actual_num_queries}.")

    idxs = np.random.choice(vectors.shape[0], size=actual_num_queries, replace=False)
    query_vectors_main = vectors[idxs] # Use a different variable name if needed to avoid conflict


    # *** Get GT distances and indices ***
    gt_distances, gt_indices = compute_ground_truth(vectors, query_vectors_main, K)

    # Base HNSW
    idx_time_base, q_time_base, avg_q_base, recall_std_base, recall_dist_base = evaluate_hnsw_base(
        vectors, query_vectors_main, gt_distances, gt_indices, K, # Pass GT distances
        M=16, ef_const=200
    )

    # HNSW with hub boosting
    # *** Choose a scaling factor for the dynamic boost ***
    # E.g., 1.0 means boost_const = hub_ratio
    # E.g., 0.5 means boost_const = 0.5 * hub_ratio
    BOOST_SCALING_FACTOR = 1.0 # You can tune this hyperparameter

    # *** Pass hubs, boost_scaling_factor, and ef_const correctly ***
    idx_time_freq, q_time_freq, avg_q_freq, recall_std_freq, recall_dist_freq = evaluate_hnsw_freq(
        vectors, query_vectors_main, gt_distances, gt_indices, K, # Pass GT distances
        hubs=hubs, boost_scaling_factor=BOOST_SCALING_FACTOR, # Pass scaling factor
        M=16, ef_const=200 # Pass ef_const
    )

    # Summary
    print("\n--- Summary ---")
    print(f"Parameters: K={K}, Num Queries={actual_num_queries}, M=16, ef_const=200, boost_scaling={BOOST_SCALING_FACTOR if 'BOOST_SCALING_FACTOR' in locals() else 'N/A'}")
    print("-" * 80)
    # Updated header to include both recalls
    print(f"Method            | Index(s) | Query(s) | AvgQuery(ms) | Recall@{K}(std) | Recall@{K}(dist)")
    print("-" * 80)
    # Updated print statements
    print(f"HNSW (base)       | {idx_time_base:<8.2f} | {q_time_base:<8.2f} | {avg_q_base*1000:<12.2f} | {recall_std_base:<13.4f} | {recall_dist_base:<14.4f}")
    print(f"HNSWfreqdistance  | {idx_time_freq:<8.2f} | {q_time_freq:<8.2f} | {avg_q_freq*1000:<12.2f} | {recall_std_freq:<13.4f} | {recall_dist_freq:<14.4f}")
    print("-" * 80)

Computing ground truth with brute-force for 40000 queries and k=100...
Done computing ground truth in 7.00s

Evaluating HNSW (base)
Index seems empty, running indexing...


HNSW (base) querying: 100%|██████████| 40000/40000 [01:39<00:00, 402.62it/s]


Index time: 1131.41s
Total query time: 99.35s, Avg/query: 2.48ms
Recall@100 (std): 0.5034
Recall@100 (dist): 0.5030

Evaluating HNSWfreqdistance
  Hub ratio: 0.0795, Calculated boost_const: 0.0795
Index seems empty, running indexing...


HNSWfreqdistance querying: 100%|██████████| 40000/40000 [02:25<00:00, 274.40it/s]


Index time: 1234.91s
Total query time: 145.77s, Avg/query: 3.64ms
Recall@100 (std): 0.3868
Recall@100 (dist): 0.3865

--- Summary ---
Parameters: K=100, Num Queries=40000, M=16, ef_const=200, boost_scaling=1.0
--------------------------------------------------------------------------------
Method            | Index(s) | Query(s) | AvgQuery(ms) | Recall@100(std) | Recall@100(dist)
--------------------------------------------------------------------------------
HNSW (base)       | 1131.41  | 99.35    | 2.48         | 0.5034        | 0.5030        
HNSWfreqdistance  | 1234.91  | 145.77   | 3.64         | 0.3868        | 0.3865        
--------------------------------------------------------------------------------
